In [50]:
# MoPhones Analysis Script
# ------------------------------------------------------------
# Purpose:
# - Load 2025 point-in-time credit snapshots
# - Enrich with DOB (age bands), Gender, Income (avg per month, income bands)
# - Derive account_status via DPD thresholds
# - Compute portfolio KPIs over time
# - Link NPS responses to nearest credit snapshot per Loan Id
# - Export CSVs and charts to ./outputs


In [51]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import load_workbook


In [52]:
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)
SNAPSHOTS = [
    ("2025-01-01", "Credit Data - 01-01-2025.csv"),
    ("2025-03-30", "Credit Data - 30-03-2025.csv"),
    ("2025-06-30", "Credit Data - 30-06-2025.csv"),
    ("2025-09-30", "Credit Data - 30-09-2025.csv"),
    ("2025-12-30", "Credit Data - 30-12-2025.csv"),
]
NPS_FILE = "NPS Data.xlsx"
SALES_CUSTOMER_FILE = "Sales and Customer Data.xlsx"


#### HELPER FUNCTIONS

In [53]:
def sheet_to_df(ws):
    """Fast convert openpyxl worksheet to pandas DataFrame (first row = header)."""
    rows = list(ws.iter_rows(values_only=True))
    header = rows[0]
    data = rows[1:]
    return pd.DataFrame(data, columns=header)



In [54]:
def income_band(series: pd.Series) -> pd.Series:
    bins = [-np.inf, 5000, 10000, 20000, 30000, 50000, 100000, 150000, np.inf]
    labels = [
        "Below 5,000",
        "5,000–9,999",
        "10,000–19,999",
        "20,000–29,999",
        "30,000–49,999",
        "50,000–99,999",
        "100,000–149,999",
        "150,000 and above",
    ]
    return pd.cut(series, bins=bins, labels=labels)


In [ ]:
def age_band(age_years: pd.Series) -> pd.Series:
    bins = [0, 18, 26, 36, 46, 56, 200]
    labels = [
        "Below 18 / Invalid",
        "18–25",
        "26–35",
        "36–45",
        "46–55",
        "Above 55",
    ]
    return pd.cut(age_years, bins=bins, labels=labels, right=False)


In [ ]:
def age_band(age_years: pd.Series) -> pd.Series:
    bins = [0, 18, 26, 36, 46, 56, 200]
    labels = [
        "Below 18 / Invalid",
        "18–25",
        "26–35",
        "36–45",
        "46–55",
        "Above 55",
    ]
    return pd.cut(age_years, bins=bins, labels=labels, right=False)


### 1) Load & combine credit snapshots


In [56]:
credit_list = []
for date_str, fname in SNAPSHOTS:
    if not Path(fname).exists():
        raise FileNotFoundError(f"Missing credit snapshot: {fname}")
    df = pd.read_csv(fname)
    df["reporting_date"] = pd.to_datetime(date_str)
    credit_list.append(df)

credit = pd.concat(credit_list, ignore_index=True)


In [57]:
# Normalize columns to uppercase with underscores
credit.rename(columns=lambda c: c.strip().upper().replace(" ", "_"), inplace=True)

In [58]:
# Type cleanup
if "DAYS_PAST_DUE" in credit.columns:
    credit["DAYS_PAST_DUE"] = pd.to_numeric(credit["DAYS_PAST_DUE"], errors="coerce").fillna(0).astype(int)
for col in ["BALANCE", "CLOSING_BALANCE", "TOTAL_PAID", "TOTAL_DUE_TODAY", "BALANCE_DUE_TO_DATE"]:
    if col in credit.columns:
        credit[col] = pd.to_numeric(credit[col], errors="coerce")

credit["REPORTING_DATE"] = pd.to_datetime(credit["REPORTING_DATE"], errors="coerce")

### 2) Load demographics & income from Sales & Customer workbook

In [59]:
if not Path(SALES_CUSTOMER_FILE).exists():
    raise FileNotFoundError(f"Missing sales & customer file: {SALES_CUSTOMER_FILE}")

wb = load_workbook(SALES_CUSTOMER_FILE, read_only=True, data_only=True)
ws_dob = wb["DOB"]          # columns include Loan Id, date_of_birth (TZ ISO)
ws_gender = wb["Gender"]    # columns include Loan Id, Gender
ws_income = wb["Income Level"]  # columns include Loan Id, Duration, Received & components

dob_df = sheet_to_df(ws_dob)
gender_df = sheet_to_df(ws_gender)
income_df = sheet_to_df(ws_income)

In [60]:
# Select/rename relevant columns
if "Loan Id" in dob_df.columns and "date_of_birth" in dob_df.columns:
    dob_df = dob_df[["Loan Id", "date_of_birth"]].rename(columns={"Loan Id": "LOAN_ID", "date_of_birth": "DOB"})
# Normalize DOB to date-only
    dob_df["DOB"] = pd.to_datetime(dob_df["DOB"].astype(str).str[:10], errors="coerce")
else:
    dob_df = pd.DataFrame(columns=["LOAN_ID", "DOB"])

if "Loan Id" in gender_df.columns and "Gender" in gender_df.columns:
    gender_df = gender_df[["Loan Id", "Gender"]].rename(columns={"Loan Id": "LOAN_ID", "Gender": "GENDER"})
else:
    gender_df = pd.DataFrame(columns=["LOAN_ID", "GENDER"])

income_cols = ["Persons Received From Total", "Banks Received", "Paybills Received", "Others", "Received", "Duration"]
available_income_cols = [c for c in income_cols if c in income_df.columns]

if "Loan Id" in income_df.columns:
    income_sel = income_df[["Loan Id"] + available_income_cols].rename(columns={"Loan Id": "LOAN_ID"})
else:
    income_sel = pd.DataFrame(columns=["LOAN_ID"] + income_cols)



In [61]:
# Numeric coercion
for c in ["Persons Received From Total", "Banks Received", "Paybills Received", "Others", "Received"]:
    if c in income_sel.columns:
        income_sel[c] = pd.to_numeric(income_sel[c], errors="coerce")

if "Duration" in income_sel.columns:
    income_sel["Duration"] = pd.to_numeric(income_sel["Duration"], errors="coerce")
else:
    income_sel["Duration"] = np.nan



In [62]:
# Total income rule:
# - If 'Received' exists, use it as total_income to avoid double counting.
# - Else sum the available components.
if "Received" in income_sel.columns:
    income_sel["TOTAL_INCOME"] = income_sel["Received"]
else:
    comp_cols = [c for c in ["Persons Received From Total", "Banks Received", "Paybills Received", "Others"] if c in income_sel.columns]
    income_sel["TOTAL_INCOME"] = income_sel[comp_cols].sum(axis=1)

income_sel["AVG_INCOME_PER_MONTH"] = income_sel["TOTAL_INCOME"] / income_sel["Duration"].replace({0: np.nan})
income_sel["INCOME_BAND"] = income_band(income_sel["AVG_INCOME_PER_MONTH"])



### 3) Join demographics & income into credit

In [63]:
credit = credit.merge(dob_df, on="LOAN_ID", how="left")
credit = credit.merge(gender_df, on="LOAN_ID", how="left")
credit = credit.merge(income_sel[["LOAN_ID", "AVG_INCOME_PER_MONTH", "INCOME_BAND"]], on="LOAN_ID", how="left")


In [68]:
# Age calculation as of REPORTING_DATE
credit["AGE_YEARS"] = ((credit["REPORTING_DATE"] - credit["DOB"]).dt.days / 365.25)
credit["AGE_BAND"] = age_band(credit["AGE_YEARS"])

### 4) Derive account_status from DPD & balance

In [69]:
# -----------------------------
# Assumptions:
# - closed: BALANCE == 0 AND CLOSING_BALANCE == 0
# - default: DAYS_PAST_DUE >= 90
# - arrears: 1 <= DAYS_PAST_DUE < 90
# - current: DAYS_PAST_DUE == 0 AND BALANCE > 0
# (if BALANCE_DUE_TO_DATE is positive, we treat that as arrears proxy in metrics)

In [70]:
status = np.where(
    (credit.get("CLOSING_BALANCE", np.nan) == 0) & (credit.get("BALANCE", np.nan) == 0),
    "closed",
    np.where(
        credit["DAYS_PAST_DUE"] >= 90,
        "default",
        np.where(
            (credit["DAYS_PAST_DUE"] > 0) & (credit["DAYS_PAST_DUE"] < 90),
            "arrears",
            np.where((credit["DAYS_PAST_DUE"] == 0) & (credit.get("BALANCE", 0) > 0), "current", "current"),
        ),
    ),
)
credit["ACCOUNT_STATUS"] = status


### 5) KPIs per reporting_date

In [ ]:
    #Arrears proxy = positive BALANCE_DUE_TO_DATE values (if present)